In [1]:
import pandas as pd
df=pd.read_csv('features_all.csv')

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes"]
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(max_depth=3, n_estimators=10),
    AdaBoostClassifier(),
    ]


In [6]:
cpc_values = df['CPC'].values

# Normalize EEG data (110 columns)
eeg_data = df.iloc[:, 7:116].values  # these columns are EEG data
scaler_eeg = MinMaxScaler()
eeg_data_normalized = scaler_eeg.fit_transform(eeg_data)
static_features = df.iloc[:, :7].values
# Normalize static features
scaler_static = MinMaxScaler()
static_features_normalized = scaler_static.fit_transform(static_features)
# Combine normalized static features, normalized EEG data, and CPC values
normalized_data = np.concatenate((static_features_normalized, eeg_data_normalized), axis=1)

# Convert CPC values to one-hot encoded labels
one_hot_labels = to_categorical(cpc_values - 1, num_classes=5)  # Assuming CPC values start from 1
# Train-test split
train_data, test_data, train_labels, test_labels = train_test_split(
    normalized_data, one_hot_labels, test_size=0.2, random_state=42
)

In [ ]:
testlab=[]
for i in test_labels.tolist():
  testlab.append([i.index(1.0)+1])
trainlab=[]
for i in train_labels.tolist():
  trainlab.append([i.index(1.0)+1])

In [ ]:
# Define model
# Define model
model1 = Sequential()
model1.add(LSTM(units=128, input_shape=(116, 1), return_sequences=True))  # Input shape should match the number of EEG columns (117)
model1.add(Dropout(0.3))  # Add dropout layer with a dropout rate of 30%
model1.add(LSTM(units=64, return_sequences=True))
model1.add(LSTM(units=32))
model1.add(Dense(units=16, activation='relu'))
model1.add(Dense(units=5, activation='softmax'))

# Compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.fit(x=train_data, y=train_labels, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 15s 669ms/step - loss: 1.5031 - accuracy: 0.3953 - val_loss: 1.3021 - val_accuracy: 0.6082
Epoch 2/50
13/13 [==============================] - 4s 322ms/step - loss: 1.2215 - accuracy: 0.5788 - val_loss: 1.1336 - val_accuracy: 0.6082
Epoch 3/50
13/13 [==============================] - 4s 319ms/step - loss: 1.1166 - accuracy: 0.5788 - val_loss: 1.0599 - val_accuracy: 0.6082
Epoch 4/50
13/13 [==============================] - 6s 430ms/step - loss: 1.0670 - accuracy: 0.5788 - val_loss: 1.0278 - val_accuracy: 0.6082
Epoch 5/50
13/13 [==============================] - 4s 348ms/step - loss: 1.0500 - accuracy: 0.5788 - val_loss: 0.9997 - val_accuracy: 0.6082
Epoch 6/50
13/13 [==============================] - 5s 411ms/step - loss: 1.0463 - accuracy: 0.5788 - val_loss: 0.9993 - val_accuracy: 0.6082
Epoch 7/50
13/13 [==============================] - 4s 322ms/step - loss: 1.0419 - accuracy: 0.5788 - val_loss: 0.9831 - val_accuracy: 0.6082
Epoch

In [ ]:
loss, accuracy = model1.evaluate(x=test_data, y=test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

4/4 [==============================] - 0s 97ms/step - loss: 0.2876 - accuracy: 0.8607
Test Loss: 0.28757545351982117, Test Accuracy: 0.8606557250022888


In [ ]:
import sklearn
print(sklearn.metrics.mean_squared_error(testlab,ypred1))
print(sklearn.metrics.mean_absolute_error(testlab,ypred1))
print(sklearn.metrics.f1_score(testlab,ypred1,average='weighted'))

0.16393442622950818
0.14754098360655737
0.8039469637830293


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import euclidean
from math import exp



gem_matrix_train = np.zeros((train_data.shape[0], train_data.shape[0]))
for i in range(train_data.shape[0]):
    for j in range(i, train_data.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(train_data[i] - train_data[j])))
        gem_matrix_train[i, j] = exp(-dtw_distance ** 2)
        gem_matrix_train[j, i] = gem_matrix_train[i, j]

gem_matrix_test = np.zeros((test_data.shape[0], train_data.shape[0]))
for i in range(test_data.shape[0]):
    for j in range(train_data.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(test_data[i] - train_data[j])))
        gem_matrix_test[i, j] = exp(-dtw_distance ** 2)

clf = SVC(kernel='precomputed')
clf.fit(gem_matrix_train, trainlab)
y_pred = clf.predict(gem_matrix_test)

accuracy = accuracy_score(testlab, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import sklearn
print(sklearn.metrics.mean_squared_error(testlab,y_pred))
print(sklearn.metrics.mean_absolute_error(testlab,y_pred))
print(sklearn.metrics.f1_score(testlab,y_pred,average='weighted'))

0.2459016393442623
0.13114754098360656
0.8840477971949977


In [ ]:
#dwt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
members=[model1,clf]


# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
  stackX = None
  for model in members:
    # make prediction
    if(model!=clf):
      yhat = model.predict(inputX)
      print('yhat: ',yhat)
      print(type(yhat))
    else:
      gem_matrix_test = np.zeros((inputX.shape[0], train_data.shape[0]))
      for i in range(inputX.shape[0]):
          for j in range(train_data.shape[0]):
              dtw_distance = np.sqrt(np.sum(np.square(inputX[i] - train_data[j])))
              gem_matrix_test[i, j] = exp(-dtw_distance ** 2)
      yhat = model.predict(gem_matrix_test)
      yhat1=[]
      for i in yhat.tolist():
        l=[0.0,0.0,0.0,0.0,0.0]
        l[i-1]=1.0
        yhat1.append(l)
      yhat=np.array(yhat1)
      print('yhat: ',yhat)
      print(type(yhat))
    # stack predictions into [rows, members, probabilities]
    if stackX is None:
      stackX = yhat
    else:
      stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
  stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
  return stackX

#[1 0 0 0 0]
#[0 1 0 0 0]

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
  # create dataset using ensemble
  stackedX = stacked_dataset(members, inputX)
  print('stacked: ',stackedX)
  # fit standalone model
  #model = LogisticRegression()
  #model=clf
  model=classifiers[0]
  #odel=knn1
  #model=rfmodel ----->give 100 percent accuracy
  model.fit(stackedX, inputy)
  return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

# fit stacked model using the ensemble
model_final = fit_stacked_model(members, test_data, testlab)
# evaluate model on test set
yhat = stacked_prediction(members, model_final, test_data)
acc = accuracy_score(testlab, yhat)
print('Stacked Test Accuracy: %.3f' % acc)



4/4 [==============================] - 0s 26ms/step
yhat:  [[0.31242415 0.06281409 0.03737791 0.01490587 0.57247794]
 [0.31245297 0.06279653 0.0373754  0.01490313 0.572472  ]
 [0.31733918 0.0631384  0.03764962 0.01494212 0.5669306 ]
 [0.3124257  0.06281271 0.03737824 0.01490691 0.5724764 ]
 [0.31247708 0.0627917  0.03737533 0.01490124 0.57245463]
 [0.3197834  0.06332499 0.03779258 0.01496618 0.56413287]
 [0.31978348 0.06332459 0.03779301 0.01496745 0.56413144]
 [0.31238985 0.06282649 0.03737965 0.01491008 0.5724939 ]
 [0.3222415  0.06350885 0.03793813 0.0149926  0.56131893]
 [0.31245926 0.06279978 0.03737575 0.01490129 0.5724639 ]
 [0.31247956 0.06279311 0.03737499 0.01489857 0.5724538 ]
 [0.31246772 0.06279684 0.03737582 0.01490115 0.5724584 ]
 [0.3122355  0.06288934 0.03739195 0.01493287 0.57255036]
 [0.31230026 0.06286221 0.03738612 0.01492255 0.5725288 ]
 [0.31236535 0.06283744 0.03738181 0.0149138  0.57250166]
 [0.31477085 0.06300502 0.03751699 0.01493548 0.5697717 ]
 [0.31970906 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4/4 [==============================] - 0s 19ms/step
yhat:  [[0.31242415 0.06281409 0.03737791 0.01490587 0.57247794]
 [0.31245297 0.06279653 0.0373754  0.01490313 0.572472  ]
 [0.31733918 0.0631384  0.03764962 0.01494212 0.5669306 ]
 [0.3124257  0.06281271 0.03737824 0.01490691 0.5724764 ]
 [0.31247708 0.0627917  0.03737533 0.01490124 0.57245463]
 [0.3197834  0.06332499 0.03779258 0.01496618 0.56413287]
 [0.31978348 0.06332459 0.03779301 0.01496745 0.56413144]
 [0.31238985 0.06282649 0.03737965 0.01491008 0.5724939 ]
 [0.3222415  0.06350885 0.03793813 0.0149926  0.56131893]
 [0.31245926 0.06279978 0.03737575 0.01490129 0.5724639 ]
 [0.31247956 0.06279311 0.03737499 0.01489857 0.5724538 ]
 [0.31246772 0.06279684 0.03737582 0.01490115 0.5724584 ]
 [0.3122355  0.06288934 0.03739195 0.01493287 0.57255036]
 [0.31230026 0.06286221 0.03738612 0.01492255 0.5725288 ]
 [0.31236535 0.06283744 0.03738181 0.0149138  0.57250166]
 [0.31477085 0.06300502 0.03751699 0.01493548 0.5697717 ]
 [0.31970906 

In [ ]:
import sklearn
print(sklearn.metrics.mean_squared_error(testlab,yhat))
print(sklearn.metrics.mean_absolute_error(testlab,yhat))
print(sklearn.metrics.f1_score(testlab,yhat,average='weighted'))

0.12295081967213115
0.07377049180327869
0.9358167218245722
